# 02 - Factor Analysis

This notebook analyzes factor returns and their properties.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.data.synthetic_data import SyntheticDataGenerator
from src.analysis.performance import PerformanceAnalyzer
from src.analysis.statistics import StatisticalTests
from src.visualization.styles import set_publication_style, PlotStyles

set_publication_style()
np.random.seed(42)

In [ ]:
# Load data
generator = SyntheticDataGenerator(seed=42)
data = generator.generate(n_months=732)

factors = data['factors']
regimes = data['regimes']
if isinstance(regimes, pd.DataFrame):
    regimes = regimes['regime']

## 1. Factor Performance Analysis

In [ ]:
# Compute performance metrics
analyzer = PerformanceAnalyzer()

metrics = {}
for col in factors.columns:
    m = analyzer.compute_metrics(factors[col])
    metrics[col] = m.to_dict()

metrics_df = pd.DataFrame(metrics).T
print("Factor Performance Metrics:")
metrics_df[['mean_return', 'volatility', 'sharpe_ratio', 'max_drawdown', 'skewness']].round(3)

In [ ]:
# Rolling Sharpe ratios
fig, ax = plt.subplots(figsize=(12, 5))

for col in factors.columns:
    rolling_sharpe = (
        factors[col].rolling(36).mean() / 
        factors[col].rolling(36).std() * np.sqrt(12)
    )
    ax.plot(rolling_sharpe.index, rolling_sharpe.values, 
            label=col, color=PlotStyles.get_factor_color(col), linewidth=1.5)

ax.axhline(y=0, color='black', linewidth=0.5)
ax.set_xlabel('Date')
ax.set_ylabel('Rolling 36-Month Sharpe Ratio')
ax.set_title('Rolling Sharpe Ratios by Factor')
ax.legend()
plt.tight_layout()
plt.show()

## 2. Factor Returns by Regime

In [ ]:
# Mean returns by regime
regime_means = pd.DataFrame()

for regime in PlotStyles.REGIME_ORDER:
    mask = regimes == regime
    if mask.sum() > 0:
        regime_means[regime] = factors.loc[mask].mean() * 100

print("Mean Monthly Returns by Regime (%):")
regime_means.round(2)

In [ ]:
# Sharpe ratios by regime
regime_sharpes = pd.DataFrame()

for regime in PlotStyles.REGIME_ORDER:
    mask = regimes == regime
    if mask.sum() > 10:
        ret = factors.loc[mask]
        sharpe = ret.mean() / ret.std() * np.sqrt(12)
        regime_sharpes[regime] = sharpe

print("\nSharpe Ratios by Regime:")
regime_sharpes.round(2)

In [ ]:
# Bar chart of returns by regime
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(PlotStyles.REGIME_ORDER))
width = 0.2

for i, factor in enumerate(factors.columns):
    vals = [regime_means.loc[factor, r] if r in regime_means.columns else 0 
            for r in PlotStyles.REGIME_ORDER]
    ax.bar(x + i*width, vals, width, label=factor,
           color=PlotStyles.get_factor_color(factor))

ax.axhline(y=0, color='black', linewidth=0.5)
ax.set_ylabel('Mean Monthly Return (%)')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels(PlotStyles.REGIME_ORDER, rotation=15, ha='right')
ax.legend()
ax.set_title('Factor Returns by Regime')
plt.tight_layout()
plt.show()

## 3. Statistical Tests

In [ ]:
# Test regime differences
tests = StatisticalTests()

test_results = []

for factor in factors.columns:
    # ANOVA across regimes
    anova = tests.test_all_regimes_equal(factors[factor], regimes)
    test_results.append({
        'Factor': factor,
        'Test': 'ANOVA',
        'Statistic': anova.statistic,
        'P-value': anova.pvalue,
        'Significant': anova.reject_null,
    })
    
    # Calm vs Crash-Spike
    if 'Calm Trend' in regimes.values and 'Crash-Spike' in regimes.values:
        diff = tests.test_regime_difference(
            factors[factor], regimes, 'Calm Trend', 'Crash-Spike'
        )
        test_results.append({
            'Factor': factor,
            'Test': 'Calm vs Crash-Spike',
            'Statistic': diff.statistic,
            'P-value': diff.pvalue,
            'Significant': diff.reject_null,
        })

pd.DataFrame(test_results).round(4)

## 4. Drawdown Analysis

In [ ]:
# Plot drawdowns
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for ax, factor in zip(axes.flatten(), factors.columns):
    dd, _ = analyzer.compute_drawdown_series(factors[factor])
    ax.fill_between(dd.index, 0, -dd.values * 100, alpha=0.7,
                   color=PlotStyles.get_factor_color(factor))
    ax.set_title(f'{factor} Drawdown')
    ax.set_ylabel('Drawdown (%)')
    ax.set_ylim([-80, 5])

plt.tight_layout()
plt.show()

## 5. Key Findings

1. **Momentum** shows the largest return variation across regimes
2. **Quality** and **Low-Risk** are more stable across market conditions
3. **Crash-Spike** states show significantly different factor behavior
4. Statistical tests confirm regime-specific return differences